In [1]:
from ortools.sat.python import cp_model
import math


In [2]:
N_CHILDREN = 24
N_WEEK_ENDS = 7

### Premier model

- une seule meute
- chaque famille a deux enfants scouts (`N_FAMILLES = N_ENFANTS / 2` )
- chaque famille a le même modèle de voiture (5pl, coffre moyen)

In [3]:
N_PLACES_PER_CAR = 5
N_FAMILIES = int(N_CHILDREN / 2)

In [4]:
model = cp_model.CpModel()

In [5]:
# Définition des variables
# C_i_j: la famille i effectue la conduite le week-end j

vars = {}
for i in range(0,N_FAMILIES):
    for j in range(0,N_WEEK_ENDS):
        vars[(i,j)] = model.NewBoolVar(f'family_{i}_weekend_{j}')

In [6]:
# Définition des contraintes

numTotalDrivings = N_WEEK_ENDS * math.ceil(N_CHILDREN / N_PLACES_PER_CAR)
minDrivingPerFamily = numTotalDrivings // N_FAMILIES
maxDrivingPerFamily = math.ceil(numTotalDrivings / N_FAMILIES)
print(f'numTotalDrivings: {numTotalDrivings}')
print(f'minDrivingPerFamily: {minDrivingPerFamily}')
print(f'maxDrivingPerFamily: {maxDrivingPerFamily}')

# Chaque famille doit faire entre minDrivingPerFamily et maxDrivingPerFamily
for i in range(0,N_FAMILIES):
    family_drivings = [vars[(i,j)] for j in range(0,N_WEEK_ENDS)]
    model.Add(sum(family_drivings) <= maxDrivingPerFamily)
    model.Add(sum(family_drivings) >= minDrivingPerFamily)

# Exactement trois conduites par week-end
for j in range(0,N_WEEK_ENDS):
    weekend_drivings = [vars[(i,j)] for i in range(0,N_FAMILIES)]
    model.Add(sum(weekend_drivings)*N_PLACES_PER_CAR >= N_CHILDREN)

numTotalDrivings: 35
minDrivingPerFamily: 2
maxDrivingPerFamily: 3


In [7]:
from driving_plan import DrivingPlan
from driving_solution_collector import DrivingSolutionCollector

In [8]:
solution_callback = DrivingSolutionCollector(limit=3, drivingVars=vars,
                                                num_families = N_FAMILIES, num_weekends = N_WEEK_ENDS)

In [9]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

solver.Solve(model, solution_callback)

2

In [10]:
solution_callback.solutions

[DrivingPlan(drivings=[(0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 5), (5, 6), (6, 0), (6, 2), (6, 6), (7, 0), (7, 1), (7, 6), (8, 1), (8, 2), (8, 6), (9, 0), (9, 1), (10, 0), (10, 1), (10, 2), (11, 0), (11, 1), (11, 2)]),
 DrivingPlan(drivings=[(0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 5), (5, 6), (6, 0), (6, 2), (6, 6), (7, 0), (7, 1), (7, 6), (8, 1), (8, 2), (8, 6), (9, 0), (9, 1), (9, 4), (10, 0), (10, 1), (10, 2), (11, 0), (11, 1), (11, 2)]),
 DrivingPlan(drivings=[(0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 5), (5, 6), (6, 0), (6, 2), (6, 6), (7, 0), (7, 1), (7, 6), (8, 1), (8, 2), (8, 6), (9, 0), (9, 1), (9, 2), (10, 0), (10, 1), (10, 2), (11, 0), (11, 1), (11, 2)])]

In [11]:
for s in solution_callback.solutions:
    s.show()

--------------------------------------------------------------------------------
Weekend #0: Famille 6, Famille 7, Famille 9, Famille 10, Famille 11
Weekend #1: Famille 7, Famille 8, Famille 9, Famille 10, Famille 11
Weekend #2: Famille 0, Famille 6, Famille 8, Famille 10, Famille 11
Weekend #3: Famille 0, Famille 2, Famille 3, Famille 4, Famille 5
Weekend #4: Famille 0, Famille 1, Famille 2, Famille 3, Famille 4
Weekend #5: Famille 1, Famille 2, Famille 3, Famille 4, Famille 5
Weekend #6: Famille 1, Famille 5, Famille 6, Famille 7, Famille 8
----------
Family #0: WE2, WE3, WE4
Family #1: WE4, WE5, WE6
Family #2: WE3, WE4, WE5
Family #3: WE3, WE4, WE5
Family #4: WE3, WE4, WE5
Family #5: WE3, WE5, WE6
Family #6: WE0, WE2, WE6
Family #7: WE0, WE1, WE6
Family #8: WE1, WE2, WE6
Family #9: WE0, WE1
Family #10: WE0, WE1, WE2
Family #11: WE0, WE1, WE2
--------------------------------------------------------------------------------
Weekend #0: Famille 6, Famille 7, Famille 9, Famille 10, Famil